# AutoKeras Tutorial + Keras Image Loading
https://autokeras.com/tutorial/image_classification/  
Felipe Giuste

In [ ]:
# ! pip install --user git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1
# ! pip install --user autokeras==1.0.4

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak

In [ ]:
# Processing/Helpers
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

# VGG16
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# User Variables

In [ ]:
## Just Examples ##
training_directory = '/data/HeartTransplant/patches_2020-06-29/'
validation_directory = '/data/HeartTransplant/patches_2020-06-29/'
testing_directory = '/data/HeartTransplant/patches_2020-06-29/'

## Image Shape ##
image_shape = (256,256,3)

## Batch Size ##
batch_size=32

# Load Dataset

In [ ]:
# Datagen for Augumentation 
train_datagen = ImageDataGenerator(
    rescale= 1./255,
#     fill_mode='wrap',
#     rotation_range=10,
#     width_shift_range=0.1, height_shift_range=0.1,
#     brightness_range=[-0.1,0.1],
#     channel_shift_range=0.1,
    horizontal_flip= True,
    vertical_flip= True,
    preprocessing_function= preprocess_input
)

valid_datagen = ImageDataGenerator(
    rescale= 1./255,
    horizontal_flip= True,
    vertical_flip= True,
    preprocessing_function= preprocess_input
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function= preprocess_input        
)


In [ ]:
# Generators for input
train_generator = train_datagen.flow_from_directory(
    training_directory,
    target_size = (image_shape[0], image_shape[1]), color_mode='rgb',
    batch_size = batch_size, 
    class_mode = "binary",
    shuffle= True,
    seed = random_state
)

validation_generator = valid_datagen.flow_from_directory(
    validation_directory,
    target_size = (image_shape[0], image_shape[1]), color_mode='rgb',
    class_mode = "binary",
    shuffle= True,
    seed = random_state)

test_generator = test_datagen.flow_from_directory(
    testing_directory,
    target_size = (image_shape[0], image_shape[1]), color_mode='rgb',
    class_mode = "binary",
    shuffle= False)

# Alternative Data Load

In [ ]:
## Load Dataset ##
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')
print(x_train.shape)  
print(y_train.shape)  
print(y_train[:3])  

# Define Classifier

In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=1)

# Train Classifier

In [ ]:
# Feed the image classifier with training data.
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)

# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

# Custom Search Space

In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=1)
clf.fit(x_train, y_train, epochs=10)